In [87]:
import requests
import json

In [88]:
API_KEY = '' #insert your own API KEY

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'uk' # uk | us | eu | au. Use comma-seperation to use multiple

MARKETS = 'h2h,spreads,totals' # h2h | spreads | totals ^

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix



In [89]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(json.dumps(odds_json, indent=4))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used']) 

Number of events: 25
[
    {
        "id": "46bdcbf099fcccf825e83c31cc8c1688",
        "sport_key": "baseball_mlb",
        "sport_title": "MLB",
        "commence_time": "2024-05-18T23:15:39Z",
        "home_team": "Texas Rangers",
        "away_team": "Los Angeles Angels",
        "bookmakers": [
            {
                "key": "paddypower",
                "title": "Paddy Power",
                "last_update": "2024-05-19T02:33:42Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-05-19T02:33:42Z",
                        "outcomes": [
                            {
                                "name": "Los Angeles Angels",
                                "price": 1.8
                            },
                            {
                                "name": "Texas Rangers",
                                "price": 1.95
                            }
                        ]
       

In [90]:
# Event object == individual sporting event
class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def num_of_outcomes(self):  # Might need to change this unsure
        bookmakers = self.data['bookmakers']
        total = []
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']
            total.extend(odds)

        num_outcomes = len(total) # I think this is currently counting every possible outcome
        return num_outcomes

    def pull_odds(self): #
        master_list = []

        bookmakers = self.data['bookmakers']
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']

            for outcome in odds:

                team_name = outcome['name']
                odds_price = outcome['price']
                bookie = bookmaker['title']

                info = {
                    "team name": team_name,
                    "odds" : odds_price,
                    "bookmaker" : bookie,
                }
                
                master_list.append(info)
        return master_list
    
    def calculate_best_odds(self, list): # | list = master_list 

        outcome_1_name = None
        outcome_2_name = None
        outcome_3_name = None

        outcome_1_best_odds = float("-inf")
        outcome_2_best_odds = float("-inf")
        outcome_3_best_odds = float("-inf")

        outcome_1_bookmaker = None
        outcome_2_bookmaker = None
        outcome_3_bookmaker = None

        for outcome in range(len(list)):
            
            if outcome_1_name is None or outcome_1_name == list[outcome]['team name']:
                outcome_1_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_1_best_odds:
                    outcome_1_best_odds = list[outcome]['odds']
                    outcome_1_bookmaker = list[outcome]['bookmaker']
            
            elif outcome_2_name is None or outcome_2_name == list[outcome]['team name']:
                outcome_2_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_2_best_odds:
                    outcome_2_best_odds = list[outcome]['odds']
                    outcome_2_bookmaker = list[outcome]['bookmaker']
            
            if outcome_3_name is None and outcome_1_name != list[outcome]['team name'] and outcome_2_name != list[outcome]['team name']:
                outcome_3_name = list[outcome]['team name']

            if outcome_3_name == list[outcome]['team name']:
                if list[outcome]['odds'] > outcome_3_best_odds:
                    outcome_3_best_odds = list[outcome]['odds']
                    outcome_3_bookmaker = list[outcome]['bookmaker']

        return [[outcome_1_name, outcome_1_best_odds, outcome_1_bookmaker], [outcome_2_name, outcome_2_best_odds, outcome_2_bookmaker], [outcome_3_name, outcome_3_best_odds, outcome_3_bookmaker]]

        # Find best odds for each outcome of a matchup and return the [Team, Odds, Bookie] for each outcome

    # Calculate arbitrage | Boolean
    def calculate_arbitrage(self, best_odds_list):
        # If only 2 outcomes remove the 3rd item
        if best_odds_list[2][0] == None:
            best_odds_list.pop(2)
        else:
            return None # Will add 3 way arb

        odds_outcome_1 = best_odds_list[0][1]
        odds_outcome_2 = best_odds_list[1][1]

        arbitrage = ((1 / odds_outcome_1) * 100) + ((1 / odds_outcome_2) * 100)
        
        return arbitrage
    
    def arbitrage_present(self, arb):
        if arb == None:
            return None # Get rid of this after adding 3 way arb
        return arb < 100
    
    def american_to_decimal(self, american_odds):
        if american_odds > 0:
            return (american_odds / 100) + 1
        elif american_odds < 0:
            return (100 / american_odds) + 1
        else:
            return None # invalid odds
        
    def decimal_to_american(self, decimal_odds):
        if decimal_odds >= 2:
            return (decimal_odds - 1) * 100
        elif 1.01 <= decimal_odds < 2:
            return -100 / (decimal_odds - 1)
        else:
            return None # invalid odds
        
    # Calculate arbitrage bet return | [Multiplier]
    def arbitrage_bet_return(self, bet_amount, arbitrage_percent):
        if arbitrage_percent == None:
            return None
        return (bet_amount / (arbitrage_percent / 100)) - bet_amount
    
    
    def calculate_bets_and_profit(self, bet_amount, best_odds_list):
        odds_outcome_1 = best_odds_list[0][1]
        odds_outcome_2 = best_odds_list[1][1]

        # Calculate the total amount to bet on each outcome
        bet_outcome_1 = (bet_amount * (1 / odds_outcome_1)) / ((1 / odds_outcome_1) + (1 / odds_outcome_2))
        bet_outcome_2 = bet_amount - bet_outcome_1

        # Calculate profit
        profit = min(bet_outcome_1 * odds_outcome_1 - bet_amount, bet_outcome_2 * odds_outcome_2 - bet_amount)

        return bet_outcome_1, bet_outcome_2, profit
    
    def send_to_discord(self, webhook_url, message):
        data = {
            "content": message
        }
        response = requests.post(webhook_url, json=data)
        if response.status_code == 204:
            print("Message sent successfully")
        else:
            print(f"Failed to send message: {response.status_code}")

In [91]:
webhook_url = '' #insert discod webhook

for i in range(len(odds_json)):
    game = Event(odds_json[i])
    master_list = game.pull_odds()
    num_outcomes = game.num_of_outcomes()
    odds = game.calculate_best_odds(master_list)
    arb = game.calculate_arbitrage(odds)

    if game.arbitrage_present(arb):
        print("Arbitrage Opportunity Found!")
        print(f"Best odds: {odds}")
        print(f"Decimal Odds: {odds[0][1]} and {odds[1][1]}")
        
        bet_outcome_1, bet_outcome_2, profit = game.calculate_bets_and_profit(100, odds)
        print(f"Bet Amounts: {bet_outcome_1:.2f} on {odds[0][0]}, {bet_outcome_2:.2f} on {odds[1][0]}")
        print(f"Profit: {profit:.2f}\n")
        
#     if game.arbitrage_present(arb):
#         print("found")
#         print(odds)
#         print(game.american_to_decimal(round(odds[0][1], 0)))
#         print(game.american_to_decimal(round(odds[1][1], 0)))
#         print(round(game.arbitrage_bet_return(100, arb), 2)) # Gives money return for $100 totbal bets
        

Arbitrage Opportunity Found!
Best odds: [['Los Angeles Angels', 2.08, 'Betfair'], ['Texas Rangers', 2.18, 'Casumo']]
Decimal Odds: 2.08 and 2.18
Bet Amounts: 51.17 on Los Angeles Angels, 48.83 on Texas Rangers
Profit: 6.44

Arbitrage Opportunity Found!
Best odds: [[None, -inf, None], [None, -inf, None]]
Decimal Odds: -inf and -inf


ZeroDivisionError: float division by zero

In [ ]:
webhook_url = '' #insert discord webhook 

for i in range(len(odds_json)):
    game = Event(odds_json[i])
    master_list = game.pull_odds()
    num_outcomes = game.num_of_outcomes()
    odds = game.calculate_best_odds(master_list)
    arb = game.calculate_arbitrage(odds)

    if game.arbitrage_present(arb):
        print("Arbitrage Opportunity Found!")
        print(f"Best odds: {odds}")
        print(f"Decimal Odds: {odds[0][1]} and {odds[1][1]}")
        
        bet_outcome_1, bet_outcome_2, profit = game.calculate_bets_and_profit(100, odds)
        print(f"Bet Amounts: {bet_outcome_1:.2f} on {odds[0][0]}, {bet_outcome_2:.2f} on {odds[1][0]}")
        print(f"Profit: {profit:.2f}")
        
        game.send_to_discord(webhook_url,
            f"Arbitrage Opportunity Found!\n"
            f"Best odds: {odds}\n"
            f"Decimal Odds: {odds[0][1]} and {odds[1][1]}\n"
            f"Bet Amounts: {bet_outcome_1:.2f} on {odds[0][0]}, {bet_outcome_2:.2f} on {odds[1][0]}\n"
            f"Profit: {profit:.2f}\n")